## Blink Features
The notebook should show how each of the features is calculated from the raw eye closure signal.
1. Test data is created
2. Blink events are identified
3. Blink events are validated visually
4. Start and Stop of each event is adjusted
5. Different Variables are calculated from eye closure signal and blink events.

In [66]:
import copy
from functools import cached_property, cache
from typing import List, Set, Dict, Tuple

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sortedcontainers import SortedSet
from tqdm import tqdm

from drowsiness_detection.data import create_eye_closure_karolinksa_dataset
from drowsiness_detection.visualize import show_frame_slider

np.random.seed(42)

In [67]:
matplotlib.use("TkAgg")  ## needed to spawn an external interactive plot

### 1. Create Test Data

In [128]:
n_samples = -1
data = next(create_eye_closure_karolinksa_dataset())
targets = data["kss"].to_numpy()
data = data["eye_closure"].to_numpy()
print(data.shape, data[:5])
data_mins = len(data) // 1800
print(f"data for {data_mins} mins.")
data_copy = data.copy()

Extracting file /home/tim/IM/data/potsdam_aeye_112020/001_1_a.json and response file: /home/tim/IM/data/sleep_alc_labels/001_1_a_karolinska.csv.
(168847,) [0.26446234 0.26638446 0.27353404 0.27267501 0.26480057]
data for 93 mins.


### 2. Identify Blink Events

Check for NaNs:

In [69]:
nan_mask = np.isnan(data)
print(sum(nan_mask))

18913


In [70]:
std = np.nanstd(data)
mean = np.nanmean(data)
print(f"{std=}, {mean=}")

std=0.09638660623372379, mean=0.29620456096475956


In [71]:
slider, ax = show_frame_slider(data=data)
ax.axhline(mean)
ax.axhline(mean + 2 * std)
ax.axhline(mean + 1 * std)
ax.axhline(mean + 0.5 * std)
plt.show(block=True)

### Validate Blink events manually
By manually looking at some blinks in the data along with different standard deviations it seems reasonable to mark every value which is 1 std. above the mean of the signal as interesting. As next step we try to find the maximum value for each group of interesting indices.

In [72]:
def blink_set_statistics(blink_sets: List[Set]):
    num_blink_events = len(blink_sets)
    mean_len = np.mean([len(s) for s in blink_sets])
    print(f"There are {num_blink_events} blink events with a mean length of {round(mean_len, 2)}.")

In [73]:
def filter_points_above_threshold(threshold: float, data: np.ndarray) -> list:
    """
    """
    above_std_sets = []
    new_set = SortedSet()
    it = np.nditer(data, flags=["f_index"])
    for value in it:
        if value > threshold:
            new_set.add(it.index)
        else:
            if new_set:
                above_std_sets.append(new_set)
                new_set = SortedSet()
    return above_std_sets


MIN_CLOSURE_FOR_BLINK = mean + 2 * std
blink_sets = filter_points_above_threshold(threshold=MIN_CLOSURE_FOR_BLINK, data=data_copy.copy())

blink_set_statistics(blink_sets)
print(f"Found an average of {len(blink_sets) // data_mins} blinks per minute.")

There are 888 blink events with a mean length of 6.21.
Found an average of 9 blinks per minute.


In [74]:
def filter_blink_sets_by_length(min_len: int, sets: list):
    return [s for s in sets if len(s) > min_len]


MIN_BLINK_LENGTH = 2
blink_sets = filter_blink_sets_by_length(min_len=MIN_BLINK_LENGTH, sets=blink_sets)
print(f"After removing blinks shorter than {MIN_BLINK_LENGTH} frames there are still a total of {len(blink_sets)} blinks.")
blink_set_statistics(blink_sets)

After removing blinks shorter than 2 frames there are still a total of 691 blinks.
There are 691 blink events with a mean length of 7.58.


Visualize the center of each blink event:

In [75]:
blink_sets_means = [np.mean(list(s)) for s in blink_sets]
matplotlib.rcParams["figure.figsize"] = 5, 10
fig, ax = plt.subplots()
ax.set_ylim(0, 2)
ax.axhline(1, lw=5)
for mean in blink_sets_means:
    ax.axvline(mean, c="r", lw=.5)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.show(block=True)

### Adjust Start and Stop of each Blink event
Because the start or stop of a blink event can be below the threshold used for finding blinks, each blink event needs to be extended such that all eye closure changes different from the resting state are included.

In [76]:
extended_blink_sets = copy.deepcopy(blink_sets)

MIN_CHANGE_OF_BLINK = .01
for index_set in extended_blink_sets:
    start, stop = index_set[0], index_set[-1]
    while (data[start] - data[start - 1]) > MIN_CHANGE_OF_BLINK:
        index_set.add(start - 1)
        start -= 1
    while (data[stop] - data[stop + 1]) > MIN_CHANGE_OF_BLINK:
        index_set.add(stop + 1)
        stop += 1

mean_len = np.mean([len(s) for s in blink_sets])
new_mean_len = np.mean([len(s) for s in extended_blink_sets])
print(f"After extending the blink event intervals the mean length changed from {round(mean_len, 2)} to {round(new_mean_len, 2)}.")
blink_set_statistics(extended_blink_sets)

After extending the blink event intervals the mean length changed from 7.58 to 12.8.
There are 691 blink events with a mean length of 12.8.


In [77]:
# # show new boundaries with horizontal lines
# slider, ax = show_frame_slider(data=data)
# for index_set in above_std_sets_extended:
#     start, stop = index_set[0], index_set[-1]
#     ax.axvline(start, c="orange")
#     ax.axvline(stop, c="orange")
# for index_set in above_std_sets:
#     index_list = list(index_set)
#     start, stop = index_set[0], index_set[-1]
#     ax.axvline(start, c="g")
#     ax.axvline(stop, c="g")
#
#
# plt.show(block=True)

### Plot old and extended boundaries of each blink event

In [78]:
# show new boundaries with green crosses
def plot_blink_sets_start_stop(*args, blink_sets: List[Set], data: np.ndarray, slider=None, ax=None, **kwargs):
    if not slider and not ax:
        slider, ax = show_frame_slider(data=data)
    boundaries = []
    for index_set in blink_sets:
        start, stop = index_set[0], index_set[-1]
        boundaries.append(start)
        boundaries.append(stop)
    values = [data[index] for index in boundaries]
    ax.plot(boundaries, values, *args, **kwargs)
    return slider, ax


slider, ax = plot_blink_sets_start_stop("bo", blink_sets=blink_sets, data=data, ms=10)

plot_blink_sets_start_stop("g+", blink_sets=extended_blink_sets, data=data, slider=slider, ax=ax, mew=3, ms=20)
plt.show(block=True)

In [79]:
blink_sets = extended_blink_sets

### Filter out blink events with an low amplitude height

In [155]:
def filter_blink_sets_by_amplitude(data: np.ndarray, blink_sets: List[Set], min_amplitude_heigth: float):
    new_blink_sets = []
    for blink_set in blink_sets:
        values = data[blink_set]
        height = max(values) - min(values)
        if height > min_amplitude_heigth:
            new_blink_sets.append(blink_set)
    return new_blink_sets


MIN_AMPLITUDE_HEIGHT = std
blink_sets_by_amplitude = filter_blink_sets_by_amplitude(data=data_copy.copy(), blink_sets=blink_sets, min_amplitude_heigth=MIN_AMPLITUDE_HEIGHT)
blink_set_statistics(blink_sets_by_amplitude)

There are 520 blink events with a mean length of 13.11.


In [156]:
slider, ax = plot_blink_sets_start_stop("bo", blink_sets=blink_sets_by_amplitude, data=data, ms=10)
plt.show(block=True)

In [157]:
blink_sets = blink_sets_by_amplitude

### Filter out blink events where first and last value are more than delta away

In [158]:
def filter_blink_sets_by_start_end_delta(data: np.ndarray, blink_sets: List[Set], max_start_end_delta: float):
    new_blink_sets = []
    for blink_set in blink_sets:
        start = data[blink_set[0]]
        end = data[blink_set[-1]]
        if abs(start - end) < max_start_end_delta:
            new_blink_sets.append(blink_set)
    return new_blink_sets


MAX_START_END_DELTA = std
blink_sets = filter_blink_sets_by_start_end_delta(data=data.copy(), blink_sets=blink_sets, max_start_end_delta=MAX_START_END_DELTA)
blink_set_statistics(blink_sets)

There are 520 blink events with a mean length of 13.11.


In [159]:
slider, ax = plot_blink_sets_start_stop("bo", blink_sets=blink_sets, data=data, ms=10)
plt.show(block=True)

### Calculate Basic Features

In [160]:
REOPENING_THRESHOLD = .05
eye_closure_data = pd.DataFrame(data_copy.copy())


class BlinkEvent:
    eye_closure_data = eye_closure_data

    def __init__(self, indices: List[int]):
        """indices contains the index of each value of one blink event. The first
        marks the start and the last one the end."""
        self.indices = indices
        self.event_data: pd.DataFrame = self.eye_closure_data.iloc[self.indices]
        self._reopen_idx = None
        self._closing_end_idx = None
        self._blink_interval = -1
        self._max_closing_speed_idx = None

    @cached_property
    def full_closure_idx(self):
        return self.event_data.idxmax(axis="index")

    @cached_property
    def full_closure(self):
        return self.event_data[self.full_closure_idx]

    @cached_property
    def amplitude(self):
        return (self.event_data.max() - self.event_data.min())  #[0]

    @cached_property
    def start_idx(self):
        return self.indices[0]

    @cached_property
    def start(self):
        return self.event_data.loc[self.start_idx, 0]

    @cached_property
    def end_idx(self):
        return self.indices[-1]

    @cached_property
    def end(self):
        return self.event_data.loc[self.end_idx, 0]

    @cached_property
    def reopen_start_idx(self):
        if self._reopen_idx:
            return self._reopen_idx
        reopen_idx = self.full_closure_idx[0]
        try:
            for idx in self.indices[self.indices.index(reopen_idx + 1):]:
                if abs(self.event_data.loc[reopen_idx, 0] - self.event_data.loc[idx, 0]) < REOPENING_THRESHOLD:
                    reopen_idx = idx
                else:
                    break
            self._reopen_idx = reopen_idx
        except ValueError as e:
            # self.plot()
            self._reopen_idx = reopen_idx

        closing_end_idx = self.full_closure_idx[0]
        try:
            for idx in self.indices[self.indices.index(reopen_idx - 1)::-1]:
                if abs(self.event_data.loc[closing_end_idx, 0] - self.event_data.loc[idx, 0]) < REOPENING_THRESHOLD:
                    closing_end_idx = idx
                else:
                    break
            self._closing_end_idx = closing_end_idx
        except ValueError as e:
            self._closing_end_idx = closing_end_idx
        return self._reopen_idx

    @cached_property
    def closing_end_idx(self):
        if self._closing_end_idx:
            return self._closing_end_idx
        _ = self.reopen_start_idx
        return self._closing_end_idx

    @cached_property
    def blink_interval(self):
        if self._blink_interval == -1:
            raise ValueError("blink interval can only be set externally.")
        return self._blink_interval

    @cached_property
    def closing_speed(self):
        time = self.closing_end_idx - self.start_idx
        if time == 0:
            return 0
        return self.amplitude[0] / time

    @cached_property
    def max_closing_speed(self):
        closing_data = self.event_data.loc[self.start_idx:self.closing_end_idx, 0]
        if len(closing_data) < 2:
            self._max_closing_speed_idx = self.closing_end_idx
            return 0
        else:
            z = closing_data.diff()
            self._max_closing_speed_idx = z[z == z.max()].index[0]
            return z.max()

    @cached_property
    def max_closing_speed_idx(self):
        if self._max_closing_speed_idx:
            return self._max_closing_speed_idx
        _ = self.max_closing_speed
        return self._max_closing_speed_idx

    @cached_property
    def blink_duration(self):
        return self.max_closing_speed_idx - self.start_idx

    @cached_property
    def lid_opening_delay(self):
        return  self.reopen_start_idx - self.closing_end_idx

    def plot(self, num_extra_frames: int = 60):
        fig, ax = plt.subplots()
        ax.plot(self.eye_closure_data[max(0, self.start_idx - num_extra_frames): min(len(self.eye_closure_data), self.end_idx + num_extra_frames)])
        ax.plot([self.start_idx, self.end_idx], [self.eye_closure_data.loc[self.start_idx, 0], self.eye_closure_data.loc[self.end_idx, 0]], "ro", ms=5)
        plt.show()


blink_events = [BlinkEvent(indices=index_set) for index_set in blink_sets]

### Add Blink Interval
The Blink Interval can only be set from outside of a BlinkEvent because it needs the end index of the previous blink event

In [161]:
def set_blink_intervals(blink_events: List[BlinkEvent]):
    for i, be in enumerate(blink_events):
        if i == 0:
            be._blink_interval = np.nan
            continue
        be._blink_interval = abs(be.start_idx - blink_events[i - 1].end_idx)
    return blink_events


blink_events = set_blink_intervals(blink_events=blink_events)

In [162]:
print(np.sum(np.isnan([be.blink_interval for be in blink_events])))
mean_blink_interal = np.nanmean([be.blink_interval for be in blink_events])
for be in blink_events:
    if np.isnan(be._blink_interval):
        be.blink_interval = mean_blink_interal
print(np.sum(np.isnan([be.blink_interval for be in blink_events])))


1
0


### Aggregate features over time interval

In [163]:
# take index of peak closure as identifier for each blink event
blink_event_dict = {be.full_closure_idx[0]: be for be in blink_events}
interval_in_min = 5
interval_in_frames = interval_in_min * (60 * 30)

In [164]:
class PreComputedSlicer:
    def __init__(self, indices: List[int], object_dict: Dict[int, object], index_interval: int):
        indices_dict = {}
        for index in indices:
            objects = []
            for key, object in object_dict.items():
                if key < index and (index - key) <= index_interval:
                    objects.append(object)
                if key > index:
                    break
            indices_dict[index] = objects
        self.indices_dict = indices_dict

    def __getitem__(self, item):
        return self.indices_dict[item]


blink_event_slicer = PreComputedSlicer(indices=eye_closure_data.index, object_dict=blink_event_dict, index_interval=interval_in_frames)

In [165]:
features_names = ["blink_duration", "blink_interval", "lid_opening_delay", "closing_speed", "max_closing_speed"]

@cache
def calculate_blink_event_statistics(events: Tuple[BlinkEvent]):
    feature_values = np.array([[be.__getattribute__(feature_name) for be in events] for feature_name in features_names], dtype=float)
    if feature_values.size == 0:
        return np.zeros( 4 * feature_values.shape[0])
    means = np.nanmean(feature_values, axis=1)
    medians = np.nanmedian(feature_values, axis=1)
    stds = np.nanstd(feature_values, axis=1)
    if np.any((stds == 0)):
        stds[(stds == 0)] = float("inf")
    skews = 3 * (means - medians) / stds
    return np.concatenate([means, medians, stds, skews])

In [169]:
feature_array = np.empty(shape=(len(eye_closure_data), len(features_names) * 4))
for index in tqdm(eye_closure_data.index[interval_in_frames:]):
    past_events = blink_event_slicer[index]
    feature_array[index] = calculate_blink_event_statistics(events=tuple(past_events))

100%|██████████| 159847/159847 [00:00<00:00, 687298.09it/s]


* naive version: 25 it/s
* with smart slicer: 25 it/s
* with aggregate operations on 2d array by removing inner for loop: 28 -40 it/s
* with cached properties for each blink event: 1750 it/s
* with caching of results for same blink events: 177618 it/s


In [170]:
feature_df = pd.DataFrame(feature_array, columns=[name + kind for kind in ["_mean", "_median", "_std", "_skew"] for name in features_names], dtype="float")

In [176]:
feature_df["kss"] = targets
data_df = feature_df.truncate(before=interval_in_frames)

In [177]:
data_df.describe()

,blink_duration_mean,blink_interval_mean,lid_opening_delay_mean,closing_speed_mean,max_closing_speed_mean,blink_duration_median,blink_interval_median,lid_opening_delay_median,closing_speed_median,max_closing_speed_median,blink_duration_std,blink_interval_std,lid_opening_delay_std,closing_speed_std,max_closing_speed_std,blink_duration_skew,blink_interval_skew,lid_opening_delay_skew,closing_speed_skew,max_closing_speed_skew
count,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00,159847.00
mean,4.22,356.73,2.29,0.16,0.28,3.39,159.56,0.73,0.16,0.27,3.76,533.38,4.47,0.08,0.11,0.49,1.05,0.98,0.22,0.28
std,1.61,137.50,1.96,0.03,0.05,1.46,112.87,0.51,0.03,0.05,3.31,257.08,4.37,0.02,0.03,0.70,0.41,0.57,0.56,0.60
min,2.53,142.84,0.35,0.05,0.13,2.00,26.00,0.00,0.03,0.10,0.64,116.25,0.48,0.04,0.04,-1.94,-0.18,-2.13,-1.32,-1.54
25%,3.29,266.72,1.00,0.15,0.26,3.00,74.00,0.00,0.15,0.25,1.09,334.38,1.68,0.06,0.10,0.31,0.79,0.70,-0.06,-0.04
50%,3.76,320.32,1.86,0.17,0.29,3.00,148.00,1.00,0.17,0.28,2.31,488.65,2.87,0.07,0.11,0.66,1.06,1.03,0.19,0.23
75%,4.48,422.91,2.40,0.18,0.31,3.00,224.50,1.00,0.18,0.31,5.50,695.33,4.39,0.08,0.13,0.89,1.36,1.37,0.57,0.53
max,11.54,1163.12,9.67,0.23,0.36,14.00,1217.00,2.00,0.22,0.36,13.05,1390.58,20.77,0.14,0.21,2.36,2.21,2.81,1.88,2.16


In [178]:
data_df.head()

,blink_duration_mean,blink_interval_mean,lid_opening_delay_mean,closing_speed_mean,max_closing_speed_mean,blink_duration_median,blink_interval_median,lid_opening_delay_median,closing_speed_median,max_closing_speed_median,...,blink_interval_std,lid_opening_delay_std,closing_speed_std,max_closing_speed_std,blink_duration_skew,blink_interval_skew,lid_opening_delay_skew,closing_speed_skew,max_closing_speed_skew,kss
9000,3.41,273.9,1.78,0.16,0.28,3.0,151.0,0.0,0.17,0.28,...,329.71,7.44,0.07,0.11,0.45,1.12,0.72,-0.52,-0.13,6
9001,3.41,273.9,1.78,0.16,0.28,3.0,151.0,0.0,0.17,0.28,...,329.71,7.44,0.07,0.11,0.45,1.12,0.72,-0.52,-0.13,6
9002,3.41,273.9,1.78,0.16,0.28,3.0,151.0,0.0,0.17,0.28,...,329.71,7.44,0.07,0.11,0.45,1.12,0.72,-0.52,-0.13,6
9003,3.41,273.9,1.78,0.16,0.28,3.0,151.0,0.0,0.17,0.28,...,329.71,7.44,0.07,0.11,0.45,1.12,0.72,-0.52,-0.13,6
9004,3.41,273.9,1.78,0.16,0.28,3.0,151.0,0.0,0.17,0.28,...,329.71,7.44,0.07,0.11,0.45,1.12,0.72,-0.52,-0.13,6


In [190]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
X = data_df[data_df.columns[:-1]].to_numpy()
Y = data_df["kss"].to_numpy().astype(int)

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [193]:
clf = SGDClassifier(alpha=0.01, max_iter=1000)
clf.fit(X_train, y_train)

SGDClassifier(alpha=0.01)

In [195]:
y_pred = clf.predict(X_test)
print(np.mean(y_pred == y_test))

0.6261421800947867


In [203]:
Y_binary = np.array(Y < 6).astype(int)

In [204]:
np.mean(Y_binary)

0.910426845671173